In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from typing import Dict, Optional, Union, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict
import tensorflow as tf
import torch
import gc, os
from tqdm import tqdm
import timm
import torch
from PIL import Image
from tqdm import tqdm
from typing import List, Tuple

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

In [3]:
class UnifiedFeatureExtractor:
    def __init__(self):
        """
        Initialize feature extractors for all models.
        """
        self.keras_extractors = {
            'resnet': self.build_resnet_feature_extractor(),
            'efficientnet': self.build_efficientnet_feature_extractor(),
            'convnext': self.build_convnext_feature_extractor()
        }
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def build_resnet_feature_extractor(self):
        base_model = tf.keras.applications.ResNet101V2(
            input_shape=(224, 224, 3), include_top=False, weights="imagenet"
        )
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        return tf.keras.Model(inputs=base_model.input, outputs=x)

    def build_efficientnet_feature_extractor(self):
        base_model = tf.keras.applications.EfficientNetB0(
            input_shape=(224, 224, 3), include_top=False, weights="imagenet"
        )
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        return tf.keras.Model(inputs=base_model.input, outputs=x)

    def build_convnext_feature_extractor(self):
        base_model = tf.keras.applications.ConvNeXtBase(
            input_shape=(224, 224, 3), include_top=False, weights="imagenet"
        )
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        return tf.keras.Model(inputs=base_model.input, outputs=x)

    def extract_features(self, df, model_name):
        """
        Extract features using models ResNet, EfficientNet and ConvNeXt.
        """
        model = self.keras_extractors[model_name]
        dataset = self.create_tf_dataset(df['image_path'].values, model_name)
        features = model.predict(dataset, verbose=1)
        column_name = f"image_features_{model_name}"
        df[column_name] = list(features)
        return df

    def create_tf_dataset(self, image_paths, model_name):
        """
        Create a TensorFlow dataset for feature extraction.
        """
        def preprocess_image(image_path):
            img_str = tf.io.read_file(image_path)
            img = tf.image.decode_jpeg(img_str, channels=3)
            img = tf.image.resize(img, [224, 224])
            if model_name == 'resnet':
                img = tf.keras.applications.resnet_v2.preprocess_input(img)
            elif model_name == 'efficientnet':
                img = tf.keras.applications.efficientnet.preprocess_input(img)
            elif model_name == 'convnext':
                img = tf.keras.applications.convnext.preprocess_input(img)
            return img

        dataset = tf.data.Dataset.from_tensor_slices(image_paths)
        dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
        return dataset

    def extract_features(self, df, model_names):
        """
        Extract features for a dataset using the models.
        """
        for model_name in model_names:
            print(f"Extracting features with {model_name}...")
            df = self.extract_features(df, model_name)
        return df

In [4]:
class FeatureExtractor:
    def __init__(self, model_name: str, device: str = None):
        """
        Initializes the feature extractor with a specified timm model.

        Parameters:
        - model_name: Name of the model (compatible with timm).
        - device: Device for computation (e.g., 'cuda' or 'cpu').
        """
        self.device = torch.device(device if device else ("cuda" if torch.cuda.is_available() else "cpu"))
        self.model = timm.create_model(model_name, pretrained=True, num_classes=0)  
        self.model.to(self.device)
        self.model.eval()

        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=False)

        if self.device.type == "cuda":
            self.model = self.model.half()  

    def preprocess_batch(self, image_paths: List[str]) -> Tuple[torch.Tensor, List[int]]:
        """
        Preprocess a batch of image paths into tensors for the model.

        Parameters:
        - image_paths: List of image file paths.

        Returns:
        - Tuple of batched tensor and a list of valid indices corresponding to the processed images.
        """
        valid_images = []
        valid_indices = []

        for idx, path in enumerate(image_paths):
            try:
                image = Image.open(path).convert("RGB")
                transformed_image = self.transforms(image)
                valid_images.append(transformed_image)
                valid_indices.append(idx)
            except Exception as e:
                warnings.warn(f"Error loading image {path}: {str(e)}")

        if not valid_images:
            return None, []

        batch_tensor = torch.stack(valid_images)
        batch_tensor = batch_tensor.to(self.device, dtype=torch.half if self.device.type == "cuda" else torch.float)
        return batch_tensor, valid_indices

    def process_batch(self, batch_tensor: torch.Tensor) -> np.ndarray:
        """
        Pass a preprocessed batch of images through the model to extract features.

        Parameters:
        - batch_tensor: A batch of preprocessed image tensors.

        Returns:
        - Numpy array of extracted features.
        """
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=self.device.type == "cuda"):
            features = self.model.forward_features(batch_tensor)
            features = self.model.forward_head(features, pre_logits=True)
        return features.cpu().numpy()

    def process_dataset(self, df, batch_size=8): 
        features_list = []
        valid_indices = []
    
        for i in tqdm(range(0, len(df), batch_size), desc="Processing images"):
            batch_paths = df['image_path'].iloc[i:i + batch_size].tolist()
            batch_tensor, batch_valid_indices = self.preprocess_batch(batch_paths)
    
            if batch_tensor is None or not batch_valid_indices:
                continue
    
            batch_features = self.process_batch(batch_tensor)
            features_list.append(batch_features)
            valid_indices.extend([i + idx for idx in batch_valid_indices])
    
            if self.device.type == "cuda":
                torch.cuda.empty_cache()
    
        all_features = np.vstack(features_list) if features_list else np.array([])
        processed_df = df.iloc[valid_indices].copy()
        return processed_df, all_features

    def __del__(self):
        """
        Destructor to clean up resources.
        """
        try:
            del self.model
            del self.transforms
            if self.device.type == "cuda":
                torch.cuda.empty_cache()
            gc.collect()
        except:
            pass

In [5]:
class UnifiedImputer:
    def __init__(self, similarity_threshold=0.6, majority_threshold=0.5):
        self.similarity_threshold = similarity_threshold
        self.majority_threshold = majority_threshold

    def compute_similarity(self, features1, features2):
        normalized_f1 = tf.nn.l2_normalize(features1, axis=1)
        normalized_f2 = tf.nn.l2_normalize(features2, axis=1)
        return tf.matmul(normalized_f1, normalized_f2, transpose_b=True)

    def impute_attributes(self, df, n_attributes):
        """
        Impute missing attributes based on visual similarity clustering.
        """
        features = np.array(list(df['image_features_resnet'].values))
        similarities = self.compute_similarity(
            tf.constant(features), tf.constant(features)
        ).numpy()
        distances = 1 - similarities
        np.fill_diagonal(distances, 0)

        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=1 - self.similarity_threshold,
            affinity="precomputed",
            linkage="complete"
        )
        clusters = clustering.fit_predict(distances)
        cluster_groups = defaultdict(list)
        for idx, cluster_id in enumerate(clusters):
            cluster_groups[cluster_id].append(idx)

        for cluster_id, indices in cluster_groups.items():
            if len(indices) > 1:
                cluster_data = df.iloc[indices]
                for attr in [f"attr_{i}" for i in range(1, n_attributes + 1)]:
                    valid_values = cluster_data[attr].dropna()
                    if valid_values.size > 0:
                        most_common_value = valid_values.mode()[0]
                        df.loc[indices, attr] = df.loc[indices, attr].fillna(most_common_value)
        return df

In [6]:
class BaseFashionModel:
    def __init__(self, num_attributes: int, attr_names: Optional[List[str]] = None):
        self.attribute_models = {}
        self.attributes = [f'attr_{i+1}' for i in range(num_attributes)]
        self.class_weights = {}
        self.n_attributes = num_attributes
        self.attr_names = attr_names if attr_names else self.attributes
        self.best_scores = {}
        self.attr_configs = {}  
        self.preprocessors = {}

    @staticmethod
    def calculate_attribute_f1_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        macro_f1 = f1_score(y_true, y_pred, average='macro')
        micro_f1 = f1_score(y_true, y_pred, average='micro')
        return 2 * (macro_f1 * micro_f1) / (macro_f1 + micro_f1)

    def calculate_score(self, y_true: pd.DataFrame, y_pred: pd.DataFrame) -> Dict[str, float]:
        scores = {}
        attribute_scores = []

        for i, attr_name in enumerate(self.attr_names):
            attr_col = f'attr_{i+1}'
            score = self.calculate_attribute_f1_score(
                y_true[attr_col].values,
                y_pred[attr_col].values
            )
            scores[attr_name] = score
            attribute_scores.append(score)

        scores['overall'] = np.mean(attribute_scores) if attribute_scores else 0.0
        return scores

    def preprocess_features(self, df: pd.DataFrame, is_training: bool = True) -> np.ndarray:
        """
        General preprocessing of image features. This implementation scales and applies PCA to all image feature columns.

        Parameters:
        - df: A pandas DataFrame containing the image feature columns.
        - is_training: If True, fit PCA and scalers; otherwise, transform using existing ones.

        Returns:
        - A numpy array containing stacked and processed feature vectors.
        """
        feature_columns = [col for col in df.columns if col.startswith("image_features_")]
        scaled_features = []
    
        for col in feature_columns:
            if is_training:
                scaler = RobustScaler()
                self.preprocessors[col] = scaler
                scaled_feature = scaler.fit_transform(np.vstack(df[col].values))
            else:
                if col not in self.preprocessors:
                    raise ValueError(f"Scaler for column '{col}' has not been fitted yet.")
                scaler = self.preprocessors[col]
                scaled_feature = scaler.transform(np.vstack(df[col].values))
    
            scaled_features.append(scaled_feature)
    
        return np.hstack(scaled_features)

    def _get_attribute_config(self, attr: str) -> dict:
        default_config = {
            'balance_strategy': 'class_weight',
            'model_params': {
                'depth': 6,
                'learning_rate': 0.1
            }
        }
        return self.attr_configs.get(attr, default_config)

    def _determine_sampling_strategy(self, y: np.ndarray) -> Dict:
        class_counts = np.bincount(y)
        max_count = np.max(class_counts)
        strategy = {i: max_count for i in range(len(class_counts))}
        return strategy

    def _apply_sampling_strategy(self, X: np.ndarray, y: np.ndarray, strategy: str, 
                               sampling_params: Optional[Dict] = None) -> tuple:
        if sampling_params is None:
            sampling_params = {}

        if strategy == 'smote':
            sampling_strategy = self._determine_sampling_strategy(y)
            sampler = SMOTE(sampling_strategy=sampling_strategy, random_state=42, **sampling_params)
            return sampler.fit_resample(X, y)
        elif strategy == 'smote_tomek':
            sampler = SMOTETomek(random_state=42)
            return sampler.fit_resample(X, y)
        elif strategy == 'undersample':
            sampling_strategy = 'auto'
            sampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42, **sampling_params)
            return sampler.fit_resample(X, y)
        else:  # No sampling
            return X, y

    def calculate_class_weights(self, y: np.ndarray, balance_strategy: str) -> Union[Dict, None]:
        if balance_strategy != 'class_weight':
            return None

        unique_classes = np.unique(y)
        if len(unique_classes) > 1:
            weights = compute_class_weight(
                class_weight='balanced',
                classes=unique_classes,
                y=y
            )
            return dict(zip(unique_classes, weights))
        return None

    def train(self, df: pd.DataFrame, validation_df: Optional[pd.DataFrame] = None, 
              epochs: int = 1000) -> 'BaseFashionModel':
        print("Preparing data for training...")
        X = self.preprocess_features(df)

        if validation_df is not None:
            X_val = self.preprocess_features(validation_df)

        print("Training models for each attribute...")
        for i, attr in enumerate(self.attributes, 1):
            print(f"\nTraining model for {attr}")

            config = self._get_attribute_config(attr)
            balance_strategy = config['balance_strategy']
            model_params = config['model_params']

            y = df[attr]
            mask = y.notna()
            X_attr = X[mask]
            y_attr = y[mask]

            X_balanced, y_balanced = self._apply_sampling_strategy(
                X_attr, y_attr, 
                balance_strategy,
                sampling_params={'k_neighbors': 5 if balance_strategy == 'smote' else None}
            )

            eval_dataset = None
            if validation_df is not None:
                y_val = validation_df[attr]
                val_mask = y_val.notna()
                X_val_attr = X_val[val_mask]
                y_val_attr = y_val[val_mask]
                eval_dataset = Pool(X_val_attr, y_val_attr)

            class_weights = self.calculate_class_weights(y_balanced, balance_strategy)

            base_params = {
                'iterations': epochs,
                'verbose': 200,
                'loss_function': 'MultiClass',
                'eval_metric': 'MultiClass',
                'custom_metric': ['F1'],
                'early_stopping_rounds': 100,
                'task_type': 'GPU',
                'nan_mode': 'Min'
            }

            model_params = {**base_params, **model_params}
            if class_weights is not None:
                model_params['class_weights'] = class_weights

            model = CatBoostClassifier(**model_params)

            if eval_dataset:
                model.fit(X_balanced, y_balanced, eval_set=eval_dataset)
            else:
                model.fit(X_balanced, y_balanced)

            self.attribute_models[attr] = model
            y_pred = model.predict(X_attr)
            score = self.calculate_attribute_f1_score(y_attr, y_pred)

            self.best_scores[attr] = score

            torch.cuda.empty_cache()
            gc.collect()
    
            print(f"Finished training {attr}. Best score: {score:.4f}")
        return self

    def predict(self, df: pd.DataFrame) -> pd.DataFrame:
        X = self.preprocess_features(df)
        results = {attr: [] for attr in self.attributes}
        
        for attr in self.attributes:
            if attr in self.attribute_models:
                model = self.attribute_models[attr]
                predictions = model.predict(X).flatten()
                results[attr] = predictions.tolist()
        
        return pd.DataFrame(results)

In [7]:
class Men_Tshirts_Model(BaseFashionModel):
    def __init__(self):
        """
        Initializes the Men T-shirts model with specific attribute configurations.
        """
        super().__init__(num_attributes=5)
        self.attr_configs = {
            'attr_1': {
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.08
                }
            },
            'attr_2': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1
                }
            },
            'attr_3': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1
                }
            },
            'attr_4': {
                'balance_strategy': 'auto',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.09
                }
            },
            'attr_5': {
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1
                }
            }
        }

In [8]:
class Sarees_Model(BaseFashionModel):
    def __init__(self):
        """
        Initializes the Sarees model with specific attribute configurations
        and preprocessing steps.
        """
        super().__init__(num_attributes=10)
        self.attr_configs = {
            'attr_1': {  
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 3,
                    'min_data_in_leaf': 10,
                    'random_strength': 1
                }
            },
            'attr_2': {  
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'min_data_in_leaf': 15
                }
            },
            'attr_3': {  
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'min_data_in_leaf': 10
                }
            },
            'attr_4': {  
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 9,
                    'learning_rate': 0.03,
                    'l2_leaf_reg': 7,
                    'min_data_in_leaf': 20
                }
            },
            'attr_5': { 
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.04,
                    'l2_leaf_reg': 5,
                    'min_data_in_leaf': 15,
                    'random_strength': 1
                }
            },
            'attr_6': {  
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'min_data_in_leaf': 10
                }
            },
            'attr_7': {  
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'min_data_in_leaf': 15
                }
            },
            'attr_8': {  
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'min_data_in_leaf': 15
                }
            },
            'attr_9': {  # print (9 classes)
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 9,
                    'learning_rate': 0.03,
                    'l2_leaf_reg': 7,
                    'min_data_in_leaf': 15,
                    'random_strength': 1
                }
            },
            'attr_10': {  
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'min_data_in_leaf': 10
                }
            }
        }

In [9]:
class Kurtis_Model(BaseFashionModel):
    def __init__(self):
        """
        Initializes the Kurtis model with specific attribute configurations.
        """
        super().__init__(num_attributes=9)
        self.attr_configs = {
            'attr_1': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'random_strength': 1
                }
            },
            'attr_2': {  # Binary classification with imbalance
                'balance_strategy': 'smote_tomek',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'random_strength': 0.8
                }
            },
            'attr_3': {  # Binary classification with imbalance
                'balance_strategy': 'smote_tomek',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'random_strength': 0.8
                }
            },
            'attr_4': {  # More balanced binary classification
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 3,
                    'random_strength': 1
                }
            },
            'attr_5': {  # Binary classification with imbalance
                'balance_strategy': 'smote_tomek',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'random_strength': 0.8
                }
            },
            'attr_6': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'random_strength': 1
                }
            },
            'attr_7': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'random_strength': 1
                }
            },
            'attr_8': {  # Sparse multi-class problem
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 7,
                    'random_strength': 1.2,
                    'min_data_in_leaf': 5
                }
            },
            'attr_9': {  # Binary classification
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 3,
                    'random_strength': 1
                }
            }
        }

In [10]:
class Women_Tshirts_Model(BaseFashionModel):
    def __init__(self):
        """
        Initializes the Women T-shirts model with specific attribute configurations.
        """
        super().__init__(num_attributes=8)
        self.attr_configs = {
            'attr_1': {  # 7-class problem with moderate confusion
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.07,
                    'l2_leaf_reg': 4,
                    'random_strength': 0.8,
                    'bootstrap_type': 'Bernoulli',
                    'subsample': 0.8
                }
            },
            'attr_2': {  # 3-class problem with strong bias
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.05,
                    'l2_leaf_reg': 5,
                    'random_strength': 1.0,
                    'bootstrap_type': 'Bernoulli',
                    'subsample': 0.85
                }
            },
            'attr_3': {  # Binary classification with class imbalance
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'random_strength': 0.5
                }
            },
            'attr_4': {  # 3-class with strong performance
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 2,
                    'random_strength': 0.3
                }
            },
            'attr_5': {  # 6-class with moderate confusion
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 9,
                    'learning_rate': 0.06,
                    'l2_leaf_reg': 5,
                    'random_strength': 1.2,
                    'bootstrap_type': 'Bernoulli',
                    'subsample': 0.75
                }
            },
            'attr_6': {  # 3-class with strong class 2
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 3,
                    'random_strength': 0.7
                }
            },
            'attr_7': {  # Binary classification with good separation
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 2,
                    'random_strength': 0.5
                }
            },
            'attr_8': {  # Binary classification with strong separation
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 5,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 2,
                    'random_strength': 0.3
                }
            }
        }

In [11]:
class Women_Tops_Model(BaseFashionModel):
    def __init__(self):
        """
        Initializes the Women Tops & Tunics model with specific attribute configurations.
        """
        super().__init__(num_attributes=10)
        self.attr_configs = {
            'attr_1': {
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.08,
                    'l2_leaf_reg': 5
                }
            },
            'attr_2': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.09,
                    'l2_leaf_reg': 3
                }
            },
            'attr_3': {
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 6,
                    'learning_rate': 0.1,
                    'l2_leaf_reg': 4  
                }
            },
            'attr_4': {
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 9,
                    'learning_rate': 0.07,
                    'random_strength': 1
                }
            },
            'attr_5': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.085,
                    'random_strength': 0.8
                }
            },
            'attr_6': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.09,
                    'l2_leaf_reg': 4
                }
            },
            'attr_7': {
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.075,
                    'random_strength': 1.2
                }
            },
            'attr_8': {
                'balance_strategy': 'smote',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.085,
                    'l2_leaf_reg': 3
                }
            },
            'attr_9': {
                'balance_strategy': 'class_weight',
                'model_params': {
                    'depth': 8,
                    'learning_rate': 0.08,
                    'random_strength': 0.9
                }
            },
            'attr_10': {
                'balance_strategy': 'hybrid',
                'model_params': {
                    'depth': 7,
                    'learning_rate': 0.09,
                    'l2_leaf_reg': 4
                }
            }
        }

In [12]:
class UnifiedFashionModelPipeline:
    def __init__(self):
        """
        Initialize the pipeline with category-specific models.
        """
        self.models = {
            'Men Tshirts': Men_Tshirts_Model(),
            'Sarees': Sarees_Model(), 
            'Kurtis': Kurtis_Model(),
            'Women Tshirts': Women_Tshirts_Model(),
            'Women Tops & Tunics': Women_Tops_Model()
        }
        self.label_encoders = {}

    def preprocess_data(self, df: pd.DataFrame, category: str, num_attributes: int) -> pd.DataFrame:
        """
        Preprocess data for a specific category, including label encoding.

        Parameters:
        - df: Input dataframe.
        - category: Category of the fashion item.
        - num_attributes: Number of attributes for the category.

        Returns:
        - Preprocessed dataframe with encoded labels.
        """
        label_encoders = {}
        for i in range(1, num_attributes + 1):
            attr = f'attr_{i}'
            le = LabelEncoder()
            df[attr] = le.fit_transform(df[attr])
            label_encoders[attr] = le
        self.label_encoders[category] = label_encoders
        return df

    def train_model(self, category: str, train_data: pd.DataFrame, val_data: pd.DataFrame, epochs: int = 2000):
        """
        Train the model for a specific category.

        Parameters:
        - category: Category of the fashion item.
        - train_data: Training dataset.
        - val_data: Validation dataset.
        - epochs: Number of epochs to train the model.
        """
        model = self.models[category]
        print(f"Training model for category: {category}")
        model.train(train_data, val_data, epochs)
        
        torch.cuda.empty_cache()
        gc.collect()
        print(f"Finished training model for {category}")
        print()

    def predict(self, test_data: pd.DataFrame, category: str) -> pd.DataFrame:
        """
        Predict attributes for a specific category.

        Parameters:
        - test_data: Test dataset containing the features.
        - category: Category of the fashion item.

        Returns:
        - DataFrame with predictions (decoded labels).
        """
        model = self.models[category]
        print(f"Predicting for category: {category}")
        predictions = model.predict(test_data)
        label_encoders = self.label_encoders[category]
        for col in predictions.columns:
            predictions[col] = label_encoders[col].inverse_transform(predictions[col])
        return predictions

    def fill_predictions(self, test_df: pd.DataFrame) -> pd.DataFrame:
        """
        Fill predictions in the test dataframe using model predictions.

        Parameters:
        - test_df: Test dataframe

        Returns:
        - Test dataframe with predictions filled.
        """
        result_df = test_df.copy()
        for category, model in self.models.items():
            category_data = test_df[test_df['Category'] == category]
            if not category_data.empty:
                predictions = self.predict(category_data, category)
                predictions = predictions.set_index(category_data.index)
                result_df.update(predictions)
        return result_df

In [ ]:
train_images = '/kaggle/input/visual-taxonomy/train_images'
data = pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')  
data['image_path'] = data['id'].apply(lambda x: os.path.join(train_images, f"{str(x).zfill(6)}.jpg"))
data = data.drop(['id'], axis=1)

feature_extractor = UnifiedFeatureExtractor()
model_names = ['resnet', 'efficientnet', 'convnext']

datasets = {
    'Men_Tshirts': data[data['Category'] == 'Men Tshirts'].reset_index(drop=True),
    'Sarees': data[data['Category'] == 'Sarees'].reset_index(drop=True),
    'Kurtis': data[data['Category'] == 'Kurtis'].reset_index(drop=True),
    'Women_Tshirts': data[data['Category'] == 'Women Tshirts'].reset_index(drop=True),
    'Womens_Tops': data[data['Category'] == 'Women Tops & Tunics'].reset_index(drop=True)
}

for model_name in model_names:
    for dataset_name, dataset_df in datasets.items():
        print(f"Processing {dataset_name} with {model_name}...")
        processed_df = feature_extractor.extract_features(dataset_df.copy(), [model_name])
        datasets[dataset_name] = processed_df
        torch.cuda.empty_cache()
        gc.collect()

imputer = UnifiedImputer()
for dataset_name, dataset_df in datasets.items():
    print(f"Imputing {dataset_name}...")
    datasets[dataset_name] = imputer.impute_attributes(dataset_df.copy(), n_attributes=10)
    gc.collect()

train_df = pd.concat(datasets.values(), ignore_index=True)

In [15]:
train_df

,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path,image_features_resnet,image_features_effnet,image_features_convnext
0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...,"[0.0, 0.14449768, 0.16205992, 0.012917416, 0.0...","[0.2860457, -0.050842, -0.1223934, -0.10037970...","[0.27109453, -0.12416643, 0.94380677, -0.18601..."
1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...,"[0.0, 0.051582135, 0.07612845, 0.0, 0.0, 0.008...","[-0.18414812, -0.1390237, -0.11305254, -0.0527...","[-0.38173568, 0.12800789, 0.40220565, 0.866345..."
2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...,"[0.0, 0.1814174, 0.048469067, 0.0, 0.0, 0.0, 0...","[0.0041561127, -0.13763672, -0.11459265, -0.08...","[-0.35483563, 0.30252552, 0.4228179, 0.8531302..."
3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...,"[0.0, 0.18438621, 0.0016736547, 0.0, 0.0, 0.08...","[-0.13240191, -0.17773184, -0.10929884, -0.072...","[-0.22742647, 0.27420285, 0.46182054, 0.543102..."
4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...,"[0.0, 0.24398537, 0.17563587, 0.0, 0.0, 0.0075...","[-0.15026966, -0.13859507, -0.117739886, -0.07...","[-0.31193736, 0.17937586, 0.3859397, 0.7873238..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70208,Women Tops & Tunics,10,multicolor,fitted,regular,square neck,casual,printed,default,short sleeves,regular sleeves,ruffles,/kaggle/input/visual-taxonomy/train_images/070...,"[0.39907238, 0.0, 0.5148285, 0.0, 0.0, 0.04164...","[-0.16685976, 0.19534639, -0.07173743, 0.06004...","[-0.29303807, -0.022686705, -0.52767247, -0.27..."
70209,Women Tops & Tunics,10,yellow,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...,"[0.0, 0.26035887, 0.0, 0.0, 0.0, 0.080200195, ...","[0.1728438, -0.019694783, -0.07842853, -0.0938...","[-0.119377755, 0.3176656, 0.60180813, 0.915949..."
70210,Women Tops & Tunics,10,maroon,fitted,crop,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...,"[0.0, 0.0, 0.03390736, 0.0, 0.0028240923, 0.02...","[-0.1257465, -0.11419905, -0.105264194, -0.110...","[-0.284321, 0.1881754, 0.6135644, -0.57130915,..."
70211,Women Tops & Tunics,10,black,regular,regular,high,casual,solid,solid,short sleeves,regular sleeves,ruffles,/kaggle/input/visual-taxonomy/train_images/070...,"[0.30429938, 0.0, 0.39254114, 0.0, 0.0, 0.0, 0...","[-0.16440184, -0.14141025, -0.1457725, 0.66374...","[-0.0926386, -0.25483623, 0.8241464, 0.1286146..."


In [14]:
pipeline = UnifiedFashionModelPipeline()
for category, model in pipeline.models.items():
    category_data = train_df[train_df['Category'] == category].copy()
    num_attributes = len(model.attributes)
    
    valid_columns = {f"attr_{i}" for i in range(1, num_attributes + 1)}
    columns_to_keep = [col for col in category_data.columns if not col.startswith("attr_") or col in valid_columns]
    category_data = category_data[columns_to_keep]
    category_data = category_data.dropna()
    category_data = pipeline.preprocess_data(category_data, category, num_attributes)
    
    train_data, val_data = train_test_split(category_data, test_size=0.2, random_state=42)
    
    pipeline.train_model(category, train_data, val_data)
    torch.cuda.empty_cache()
    gc.collect()

Training model for category: Men Tshirts
Preparing data for training...
Training models for each attribute...

Training model for attr_1
0:	learn: 1.3060559	test: 1.3166457	best: 1.3166457 (0)	total: 12.5s	remaining: 6h 55m 35s
200:	learn: 0.2957116	test: 0.6531640	best: 0.6531640 (200)	total: 37.8s	remaining: 5m 38s
400:	learn: 0.2058022	test: 0.6410570	best: 0.6402187 (385)	total: 1m	remaining: 4m
bestTest = 0.6402186741
bestIteration = 385
Shrink model to first 386 iterations.
Finished training attr_1. Best score: 0.9526

Training model for attr_2
0:	learn: 0.6083637	test: 0.6088103	best: 0.6088103 (0)	total: 220ms	remaining: 7m 18s
200:	learn: 0.0134421	test: 0.0399376	best: 0.0399376 (200)	total: 4.57s	remaining: 40.9s
400:	learn: 0.0069285	test: 0.0376417	best: 0.0376223 (399)	total: 8.87s	remaining: 35.4s
600:	learn: 0.0048327	test: 0.0371646	best: 0.0369658 (593)	total: 13.1s	remaining: 30.4s
800:	learn: 0.0037987	test: 0.0367456	best: 0.0366466 (782)	total: 17.2s	remaining: 25

In [ ]:
test_images = '/kaggle/input/visual-taxonomy/test_images'
test_df = pd.read_csv('/kaggle/input/visual-taxonomy/test.csv')  
test_df['image_path'] = test_df['id'].apply(lambda x: os.path.join(test_images, f"{str(x).zfill(6)}.jpg"))
test_predictions = pipeline.fill_predictions(test_df)
test_predictions = test_predictions[['id', 'Category', 'len', 'attr_1', 'attr_2', 'attr_3', 'attr_4',\
                                     'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']]